In [2]:
import pandas as pd

In [4]:
data_5s = pd.read_json("data_5scheduler.json")
data_5s.head(10)

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd,300,[Staff],,,,False,0
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd,300,[Anup Gampa],,,,True,0
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...,HarveyMudd,300,[Balseiro],,,,False,0
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...,HarveyMudd,300,[de Laet],,,,False,0
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...,HarveyMudd,300,[Marianne De Laet],,,,True,0
5,War and Conflict,ANTH-115-HM,“The wings of the butterfly—that cause the hur...,HarveyMudd,300,[de Laet],,,,False,0
6,Rationalities,ANTH-134-HM,What does it mean to be rational? Does it mean...,HarveyMudd,300,[de Laet],Offered alternate years,Any introductory course in anthropology or any...,,False,0
7,A History of Landscape Photography,ARHI-131-HM,This course explores how photographic landscap...,HarveyMudd,300,[Fandell],,,,False,0
8,Modern and Contemporary Art Practices,ART-002-HM,This class is an experimental lecture style ar...,HarveyMudd,300,[Fandell],,,,False,0
9,Photography,ART-033-HM,Approaching the medium from an artistic perspe...,HarveyMudd,300,[Fandell],,ART002 HM,,False,150


In [5]:
data_5s.describe()

,credits,fee
count,4443.000000,4443.000000
mean,85.253657,0.082152
std,70.623956,2.864374
min,0.000000,0.000000
25%,25.000000,0.000000
50%,100.000000,0.000000
75%,100.000000,0.000000
max,600.000000,150.000000


In [8]:
data_stripped = data_5s.drop(["source", "credits", "instructors", "offered", "prerequisites", "corequisites", "currently_offered", "fee"], axis=1)

In [9]:
data_stripped.head()

,title,identifier,description
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...


In [27]:
from nltk.tokenize import word_tokenize

In [36]:
data_tokenized = data_stripped.copy()
data_tokenized["title_tokenized"] = data_tokenized.title.apply(lambda x: [i.lower() for i in word_tokenize(x)])
data_tokenized["description_tokenized"] = data_tokenized.description.apply(lambda x: [i.lower() for i in word_tokenize(x)])
data_tokenized["tokens"] = data_tokenized.title_tokenized + data_tokenized.description_tokenized
data_tokenized.drop(["title", "description", "title_tokenized", "description_tokenized"], axis=1, inplace=True)
data_tokenized.head()

,identifier,tokens
0,AMST-103-HM,"[introduction, to, american, cultures, an, int..."
1,AMST-115-HM,"[print, and, american, culture, covers, numero..."
2,AMST-120-HM,"[hyphenated, americans, a, focus, on, the, exp..."
3,ANTH-110-HM,"[life, :, knowledge, ,, belief, ,, and, cultur..."
4,ANTH-111-HM,"[introduction, to, the, anthropology, of, scie..."


In [54]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=data_tokenized.tokens, min_count=2)
print(model.wv)

KeyedVectors<vector_size=100, 9543 keys>


In [65]:
def get_average_vector(tokens):
    vectors = [model.wv[i] for i in tokens if i in model.wv.key_to_index]
    average_vector = sum(vectors)/len(vectors)
    return average_vector

data_vectorized = data_tokenized.copy()
data_vectorized["vector"] = data_vectorized.tokens.apply(get_average_vector)
data_vectorized.drop("tokens", axis=1, inplace=True)
data_vectorized.head()

,identifier,vector
0,AMST-103-HM,"[-0.8944389, 0.4807644, 0.18470237, 0.26499087..."
1,AMST-115-HM,"[-0.3409733, 0.09692527, 0.18012655, 0.3223548..."
2,AMST-120-HM,"[-0.5915687, 0.22977571, 0.12964556, 0.4453324..."
3,ANTH-110-HM,"[-0.46314567, 0.26716194, 0.16854253, 0.531863..."
4,ANTH-111-HM,"[-0.58057773, 0.33517152, 0.16949482, 0.441985..."


In [79]:
data_vectorized[data_vectorized['identifier'] == 'AMST-103-HM'].loc[0]

identifier                                          AMST-103-HM
vector        [-0.8944389, 0.4807644, 0.18470237, 0.26499087...
Name: 0, dtype: object

In [100]:
import numpy as np

def get_similar_courses(search_course, data_vectorized, data_5s):
    this_vector = data_vectorized[data_vectorized['identifier'] == 'AMST-103-HM'].loc[0].vector
    data_search = data_vectorized[data_vectorized.identifier != search_course]
    data_info = data_5s.copy()
    data_info["cosine_sim"] = data_search.apply(lambda x: np.inner(x.vector, this_vector) / (np.linalg.norm(x.vector) * np.linalg.norm(this_vector)), axis=1)
    data_info.sort_values("cosine_sim", inplace=True, ascending=False)
    return data_info.head(10)

In [101]:
get_similar_courses("AMST-103-HM", data_vectorized, data_5s)

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,cosine_sim
4088,Introduction to Sound Studies,MS-052-PZ,This is an introductory level course exploring...,Pitzer,0,[],,,,False,0,0.987862
3489,Introduction to the History of Judaism and Jew...,REL-310-CG,See Claremont Graduate University catalog for ...,Scripps,100,[],,,,False,0,0.972862
90,Lasers in Chemistry,CHEM-168-HM,Introduction to principles of lasers and laser...,HarveyMudd,200,[Staff],Fall and Spring,CHEM051 HM and PHYS051 HM,Juniors and seniors only.,False,0,0.972363
3832,Introduction to African American Literature Be...,ENGL-012A-AF,"This course is a survey of major periods, auth...",Pitzer,100,[Laura Harris],,,,True,0,0.968977
4099,Introduction to Video Art Lab,MS-082L-PZ,Lab component to Introduction to Video Art. Re...,Pitzer,100,[Carlin Wing],,,,True,0,0.968968
357,Introduction to Psychology,PSYC-053-HM,An introduction to the field of psychology wit...,HarveyMudd,300,[Staff],,,,False,0,0.968369
3594,Introduction to Archaeology and Biological Ant...,ANTH-001-PZ,"An introduction to the basic concepts, theorie...",Pitzer,0,[Cory Kohn],,,,True,0,0.967316
86,Advanced Physical Chemistry: Electronic Struct...,CHEM-164-HM,An examination of modern methods for approxima...,HarveyMudd,200,[Staff],,CHEM052 HM and PHYS051 HM,,False,0,0.966917
1259,Foundations of 2D Design,ART-021-PO,Foundations of 2D Design is a hands on introdu...,Pomona,100,[M. Allen],Each semester.,,,False,0,0.965521
3562,Introduction to Women and Gender Studies,WGS-301-CG,See the Claremont Graduate University Catalog ...,Scripps,100,[],,,,False,0,0.965091
